In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn_genetic import GAFeatureSelectionCV
from sklearn_genetic.plots import plot_fitness_evolution
from sklearn.metrics import roc_auc_score

from collections import Counter

# --- Load Data ---
data = pd.read_csv('radiomics_features_washu2_p1_143_with_labels_sdf3_nd_normseg.csv') 
X = data.drop(columns=['PatientID', 'GT', 'ImagePath', 'Patient ID'])
y = (data['GT']<1).astype('int')

# --- Split ---
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# # --- Define multiple estimators ---
# models = {
#     "RandomForest": RandomForestClassifier(random_state=42),
#     "LogisticRegression": LogisticRegression(max_iter=1000),
#     "DecisionTree": DecisionTreeClassifier(random_state=42),
#     "SVM": SVC(kernel='linear', probability=True)
# }

models = {
    "RandomForest": RandomForestClassifier(random_state=42),
    "LogisticRegression": LogisticRegression(max_iter=1000),
    "DecisionTree": DecisionTreeClassifier(random_state=42),
    "GradientBoosting": GradientBoostingClassifier(random_state=42),  # ✅ Added
    "ExtraTrees": ExtraTreesClassifier(random_state=42),              # ✅ Added
}



# --- Run GA Feature Selection for each model ---
selected_features_per_model = {}

# --- Run GA Feature Selection for each model ---
for name, model in models.items():
    print(f"\n🧬 Running GA Feature Selection with {name}")
    
    selector = GAFeatureSelectionCV(
        estimator=model,
        cv=5,
        scoring="roc_auc",  # ✅ AUC instead of accuracy
        population_size=30,
        generations=20,
        n_jobs=-1,
        verbose=False,
        keep_top_k=5,
        crossover_probability=0.8,
        mutation_probability=0.2,
        elitism=True,
        algorithm='eaMuPlusLambda'
    )
    
    selector.fit(X_train, y_train)
    selected = X_train.columns[selector.support_].tolist()
    selected_features_per_model[name] = selected

    # Evaluate on held-out test set using AUC
    X_train_sel = X_train[selected]
    X_test_sel = X_test[selected]
    
    model.fit(X_train_sel, y_train)
    y_pred_proba = model.predict_proba(X_test_sel)[:, 1]  # 🔥 Probability for positive class
    
    auc_score = roc_auc_score(y_test, y_pred_proba)
    
    print(f"✅ {name} selected features ({len(selected)}): {selected}")
    print(f"   Test AUC: {auc_score:.4f}")


c:\Users\mdiqb\anaconda3\envs\py312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



🧬 Running GA Feature Selection with RandomForest
✅ RandomForest selected features (231): ['Unnamed: 0', 'original_shape2D_Elongation_x', 'original_shape2D_MajorAxisLength_x', 'original_shape2D_MaximumDiameter_x', 'original_shape2D_MinorAxisLength_x', 'original_shape2D_Perimeter_x', 'original_shape2D_PerimeterSurfaceRatio_x', 'original_shape2D_PixelSurface_x', 'original_firstorder_Entropy_x', 'original_firstorder_Maximum_x', 'original_firstorder_Mean_x', 'original_firstorder_Range_x', 'original_firstorder_RobustMeanAbsoluteDeviation_x', 'original_firstorder_Skewness_x', 'original_glcm_ClusterTendency_x', 'original_glcm_Contrast_x', 'original_glcm_DifferenceVariance_x', 'original_glcm_Id_x', 'original_glcm_Idmn_x', 'original_glcm_Idn_x', 'original_glcm_InverseVariance_x', 'original_glcm_MCC_x', 'original_glcm_MaximumProbability_x', 'original_glcm_SumSquares_x', 'original_glrlm_GrayLevelNonUniformity_x', 'original_glrlm_HighGrayLevelRunEmphasis_x', 'original_glrlm_LongRunLowGrayLevelEmph

c:\Users\mdiqb\anaconda3\envs\py312\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\mdiqb\anaconda3\envs\py312\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logisti

✅ LogisticRegression selected features (230): ['Unnamed: 0', 'original_shape2D_Elongation_x', 'original_shape2D_MeshSurface_x', 'original_shape2D_Perimeter_x', 'original_shape2D_PerimeterSurfaceRatio_x', 'original_shape2D_PixelSurface_x', 'original_firstorder_90Percentile_x', 'original_firstorder_Entropy_x', 'original_firstorder_InterquartileRange_x', 'original_firstorder_Median_x', 'original_firstorder_Minimum_x', 'original_firstorder_Skewness_x', 'original_firstorder_Uniformity_x', 'original_firstorder_Variance_x', 'original_glcm_Autocorrelation_x', 'original_glcm_Contrast_x', 'original_glcm_Correlation_x', 'original_glcm_DifferenceAverage_x', 'original_glcm_DifferenceEntropy_x', 'original_glcm_Idmn_x', 'original_glcm_Imc2_x', 'original_glcm_InverseVariance_x', 'original_glcm_JointAverage_x', 'original_glcm_JointEntropy_x', 'original_glcm_SumAverage_x', 'original_glcm_SumSquares_x', 'original_glrlm_GrayLevelNonUniformity_x', 'original_glrlm_GrayLevelNonUniformityNormalized_x', 'origi

In [2]:
# --- Combine feature selections ---
all_selected = [f for features in selected_features_per_model.values() for f in features]
feature_votes = Counter(all_selected)

print("\n🗳️ Feature vote count across models:")
for feat, count in feature_votes.most_common():
    print(f"  {feat}: {count} votes")

# --- Final selection based on vote threshold (e.g., at least 2 models agree) ---
consensus_features = [feat for feat, count in feature_votes.items() if count >= 3]

print("\n✅ Final consensus features selected by at least 2 models:")
print(consensus_features)



🗳️ Feature vote count across models:
  Unnamed: 0: 5 votes
  original_shape2D_PixelSurface_x: 5 votes
  original_glcm_SumSquares_x: 5 votes
  original_glrlm_HighGrayLevelRunEmphasis_x: 5 votes
  wavelet-H_firstorder_Energy_x: 5 votes
  wavelet-H_glcm_JointAverage_x: 5 votes
  wavelet-H_glszm_SmallAreaHighGrayLevelEmphasis_x: 5 votes
  wavelet-L_glrlm_ShortRunHighGrayLevelEmphasis_x: 5 votes
  original_shape2D_Elongation_y: 5 votes
  original_glcm_SumEntropy_y: 5 votes
  original_glrlm_RunEntropy_y: 5 votes
  wavelet-H_glcm_ClusterTendency_y: 5 votes
  wavelet-H_glrlm_HighGrayLevelRunEmphasis_y: 5 votes
  wavelet-H_glrlm_ShortRunLowGrayLevelEmphasis_y: 5 votes
  wavelet-H_glszm_SmallAreaHighGrayLevelEmphasis_y: 5 votes
  wavelet-L_glcm_Idm_y: 5 votes
  wavelet-L_glrlm_RunVariance_y: 5 votes
  wavelet-L_glrlm_ShortRunLowGrayLevelEmphasis_y: 5 votes
  wavelet-L_glszm_LowGrayLevelZoneEmphasis_y: 5 votes
  original_shape2D_MaximumDiameter_x: 4 votes
  original_shape2D_PerimeterSurfaceRatio

In [3]:
len(['original_shape2D_Elongation', 'original_shape2D_MajorAxisLength', 'original_shape2D_MeshSurface', 'original_shape2D_Perimeter', 'original_shape2D_PerimeterSurfaceRatio', 'original_shape2D_PixelSurface', 'original_firstorder_10Percentile', 'original_firstorder_90Percentile', 'original_firstorder_Entropy', 'original_firstorder_Kurtosis', 'original_firstorder_Median', 'original_firstorder_Range', 'original_firstorder_RootMeanSquared', 'original_firstorder_Skewness', 'original_firstorder_Uniformity', 'original_firstorder_Variance', 'original_glcm_Contrast', 'original_glcm_DifferenceAverage', 'original_glcm_DifferenceEntropy', 'original_glcm_DifferenceVariance', 'original_glcm_Id', 'original_glcm_Idm', 'original_glcm_Idmn', 'original_glcm_Imc1', 'original_glcm_JointEnergy', 'original_glcm_SumEntropy', 'original_glrlm_GrayLevelNonUniformity', 'original_glrlm_GrayLevelVariance', 'original_glrlm_HighGrayLevelRunEmphasis', 'original_glrlm_LongRunEmphasis', 'original_glrlm_LowGrayLevelRunEmphasis', 'original_glrlm_RunEntropy', 'original_glrlm_RunLengthNonUniformityNormalized', 'original_glrlm_RunPercentage', 'original_glrlm_RunVariance', 'original_glrlm_ShortRunEmphasis', 'original_glrlm_ShortRunLowGrayLevelEmphasis', 'original_glszm_GrayLevelNonUniformityNormalized', 'original_glszm_HighGrayLevelZoneEmphasis', 'original_glszm_LargeAreaHighGrayLevelEmphasis', 'original_glszm_SizeZoneNonUniformityNormalized', 'original_glszm_SmallAreaEmphasis', 'original_glszm_SmallAreaLowGrayLevelEmphasis', 'wavelet-H_firstorder_10Percentile', 'wavelet-H_firstorder_90Percentile', 'wavelet-H_firstorder_Energy', 'wavelet-H_firstorder_Median', 'wavelet-H_firstorder_Minimum', 'wavelet-H_firstorder_RobustMeanAbsoluteDeviation', 'wavelet-H_firstorder_RootMeanSquared', 'wavelet-H_glcm_Autocorrelation', 'wavelet-H_glcm_ClusterProminence', 'wavelet-H_glcm_ClusterTendency', 'wavelet-H_glcm_Contrast', 'wavelet-H_glcm_Idm', 'wavelet-H_glcm_Idmn', 'wavelet-H_glcm_Imc1', 'wavelet-H_glcm_Imc2', 'wavelet-H_glcm_JointAverage', 'wavelet-H_glcm_JointEnergy', 'wavelet-H_glcm_SumAverage', 'wavelet-H_glrlm_GrayLevelNonUniformity', 'wavelet-H_glrlm_HighGrayLevelRunEmphasis', 'wavelet-H_glrlm_LongRunEmphasis', 'wavelet-H_glrlm_ShortRunEmphasis', 'wavelet-H_glrlm_ShortRunHighGrayLevelEmphasis', 'wavelet-H_glrlm_ShortRunLowGrayLevelEmphasis', 'wavelet-H_glszm_GrayLevelNonUniformity', 'wavelet-H_glszm_GrayLevelVariance', 'wavelet-H_glszm_LargeAreaEmphasis', 'wavelet-H_glszm_LargeAreaHighGrayLevelEmphasis', 'wavelet-H_glszm_LargeAreaLowGrayLevelEmphasis', 'wavelet-H_glszm_SizeZoneNonUniformityNormalized', 'wavelet-H_glszm_ZoneVariance', 'wavelet-L_firstorder_Kurtosis', 'wavelet-L_firstorder_Maximum', 'wavelet-L_firstorder_Mean', 'wavelet-L_firstorder_Range', 'wavelet-L_firstorder_Skewness', 'wavelet-L_firstorder_TotalEnergy', 'wavelet-L_firstorder_Uniformity', 'wavelet-L_glcm_DifferenceAverage', 'wavelet-L_glcm_DifferenceEntropy', 'wavelet-L_glcm_DifferenceVariance', 'wavelet-L_glcm_Id', 'wavelet-L_glcm_Idn', 'wavelet-L_glcm_Imc2', 'wavelet-L_glcm_JointEnergy', 'wavelet-L_glcm_MCC', 'wavelet-L_glcm_MaximumProbability', 'wavelet-L_glcm_SumAverage', 'wavelet-L_glcm_SumSquares', 'wavelet-L_glrlm_GrayLevelNonUniformity', 'wavelet-L_glrlm_GrayLevelNonUniformityNormalized', 'wavelet-L_glrlm_HighGrayLevelRunEmphasis', 'wavelet-L_glrlm_LongRunHighGrayLevelEmphasis', 'wavelet-L_glrlm_RunLengthNonUniformityNormalized', 'wavelet-L_glrlm_ShortRunEmphasis', 'wavelet-L_glrlm_ShortRunHighGrayLevelEmphasis', 'wavelet-L_glszm_GrayLevelNonUniformityNormalized', 'wavelet-L_glszm_LargeAreaHighGrayLevelEmphasis', 'wavelet-L_glszm_LargeAreaLowGrayLevelEmphasis', 'wavelet-L_glszm_SizeZoneNonUniformityNormalized', 'wavelet-L_glszm_SmallAreaHighGrayLevelEmphasis', 'wavelet-L_glszm_SmallAreaLowGrayLevelEmphasis', 'wavelet-L_glszm_ZoneVariance', 'original_shape2D_Sphericity', 'original_firstorder_Energy', 'original_firstorder_TotalEnergy', 'original_glcm_ClusterProminence', 'original_glcm_ClusterTendency', 'original_glcm_Correlation', 'original_glcm_JointEntropy', 'original_glcm_MCC', 'original_glcm_MaximumProbability', 'original_glrlm_GrayLevelNonUniformityNormalized', 'original_glrlm_LongRunLowGrayLevelEmphasis', 'original_glszm_LargeAreaLowGrayLevelEmphasis', 'original_glszm_ZoneEntropy', 'wavelet-H_firstorder_InterquartileRange', 'wavelet-H_firstorder_Maximum', 'wavelet-H_firstorder_Variance', 'wavelet-H_glcm_Correlation', 'wavelet-H_glcm_DifferenceEntropy', 'wavelet-H_glcm_DifferenceVariance', 'wavelet-H_glcm_InverseVariance', 'wavelet-H_glcm_JointEntropy', 'wavelet-H_glcm_SumEntropy', 'wavelet-H_glcm_SumSquares', 'wavelet-H_glrlm_GrayLevelNonUniformityNormalized', 'wavelet-H_glrlm_LongRunHighGrayLevelEmphasis', 'wavelet-H_glrlm_RunLengthNonUniformity', 'wavelet-H_glszm_HighGrayLevelZoneEmphasis', 'wavelet-H_glszm_SizeZoneNonUniformity', 'wavelet-H_glszm_SmallAreaEmphasis', 'wavelet-H_glszm_SmallAreaLowGrayLevelEmphasis', 'wavelet-H_glszm_ZoneEntropy', 'wavelet-H_glszm_ZonePercentage', 'wavelet-L_firstorder_10Percentile', 'wavelet-L_firstorder_90Percentile', 'wavelet-L_firstorder_Energy', 'wavelet-L_firstorder_Entropy', 'wavelet-L_firstorder_InterquartileRange', 'wavelet-L_firstorder_Variance', 'wavelet-L_glcm_ClusterProminence', 'wavelet-L_glcm_Correlation', 'wavelet-L_glcm_Idm', 'wavelet-L_glcm_InverseVariance', 'wavelet-L_glcm_JointAverage', 'wavelet-L_glrlm_GrayLevelVariance', 'wavelet-L_glrlm_RunLengthNonUniformity', 'original_shape2D_MinorAxisLength', 'original_firstorder_InterquartileRange', 'original_firstorder_RobustMeanAbsoluteDeviation', 'original_glcm_ClusterShade', 'original_glcm_InverseVariance', 'original_glcm_JointAverage', 'original_glrlm_ShortRunHighGrayLevelEmphasis', 'original_glszm_SmallAreaHighGrayLevelEmphasis', 'original_glszm_ZoneVariance', 'wavelet-H_glcm_ClusterShade', 'wavelet-H_glcm_MaximumProbability', 'wavelet-H_glrlm_LongRunLowGrayLevelEmphasis', 'wavelet-H_glrlm_RunEntropy', 'wavelet-L_firstorder_Median', 'wavelet-L_glcm_Imc1', 'wavelet-L_glcm_SumEntropy', 'wavelet-L_glszm_GrayLevelNonUniformity', 'wavelet-L_glszm_HighGrayLevelZoneEmphasis', 'wavelet-L_glszm_LargeAreaEmphasis', 'wavelet-L_glszm_LowGrayLevelZoneEmphasis', 'wavelet-L_glszm_SizeZoneNonUniformity', 'wavelet-L_glszm_SmallAreaEmphasis', 'original_glcm_SumAverage', 'original_glszm_GrayLevelNonUniformity', 'original_glszm_LargeAreaEmphasis', 'wavelet-H_glcm_MCC', 'wavelet-L_glcm_ClusterTendency', 'wavelet-L_glcm_Contrast', 'wavelet-L_glrlm_LongRunEmphasis', 'wavelet-L_glrlm_RunPercentage']
)

181

In [5]:
len(['original_shape2D_Elongation', 'original_shape2D_MajorAxisLength', 'original_shape2D_Perimeter', 'original_shape2D_PerimeterSurfaceRatio', 'original_shape2D_PixelSurface', 'original_firstorder_10Percentile', 'original_firstorder_90Percentile', 'original_firstorder_Entropy', 'original_firstorder_Kurtosis', 'original_firstorder_Median', 'original_firstorder_Range', 'original_firstorder_RootMeanSquared', 'original_glcm_Contrast', 'original_glcm_DifferenceAverage', 'original_glcm_DifferenceEntropy', 'original_glcm_DifferenceVariance', 'original_glcm_Id', 'original_glcm_Idm', 'original_glcm_Imc1', 'original_glcm_JointEnergy', 'original_glcm_SumEntropy', 'original_glrlm_GrayLevelNonUniformity', 'original_glrlm_GrayLevelVariance', 'original_glrlm_HighGrayLevelRunEmphasis', 'original_glrlm_LongRunEmphasis', 'original_glrlm_LowGrayLevelRunEmphasis', 'original_glrlm_RunLengthNonUniformityNormalized', 'original_glrlm_RunVariance', 'original_glrlm_ShortRunEmphasis', 'original_glrlm_ShortRunLowGrayLevelEmphasis', 'original_glszm_HighGrayLevelZoneEmphasis', 'original_glszm_LargeAreaHighGrayLevelEmphasis', 'original_glszm_SizeZoneNonUniformityNormalized', 'original_glszm_SmallAreaEmphasis', 'original_glszm_SmallAreaLowGrayLevelEmphasis', 'wavelet-H_firstorder_90Percentile', 'wavelet-H_firstorder_Energy', 'wavelet-H_firstorder_Median', 'wavelet-H_firstorder_Minimum', 'wavelet-H_firstorder_RootMeanSquared', 'wavelet-H_glcm_Autocorrelation', 'wavelet-H_glcm_ClusterProminence', 'wavelet-H_glcm_ClusterTendency', 'wavelet-H_glcm_Idm', 'wavelet-H_glcm_Idmn', 'wavelet-H_glcm_Imc1', 'wavelet-H_glcm_Imc2', 'wavelet-H_glcm_JointAverage', 'wavelet-H_glcm_JointEnergy', 'wavelet-H_glcm_SumAverage', 'wavelet-H_glrlm_GrayLevelNonUniformity', 'wavelet-H_glrlm_LongRunEmphasis', 'wavelet-H_glrlm_ShortRunEmphasis', 'wavelet-H_glrlm_ShortRunHighGrayLevelEmphasis', 'wavelet-H_glrlm_ShortRunLowGrayLevelEmphasis', 'wavelet-H_glszm_GrayLevelNonUniformity', 'wavelet-H_glszm_LargeAreaEmphasis', 'wavelet-H_glszm_LargeAreaHighGrayLevelEmphasis', 'wavelet-H_glszm_SizeZoneNonUniformityNormalized', 'wavelet-L_firstorder_Kurtosis', 'wavelet-L_firstorder_Maximum', 'wavelet-L_firstorder_Mean', 'wavelet-L_firstorder_Skewness', 'wavelet-L_firstorder_TotalEnergy', 'wavelet-L_firstorder_Uniformity', 'wavelet-L_glcm_DifferenceAverage', 'wavelet-L_glcm_DifferenceEntropy', 'wavelet-L_glcm_DifferenceVariance', 'wavelet-L_glcm_Id', 'wavelet-L_glcm_Idn', 'wavelet-L_glcm_Imc2', 'wavelet-L_glcm_JointEnergy', 'wavelet-L_glcm_MCC', 'wavelet-L_glcm_MaximumProbability', 'wavelet-L_glcm_SumSquares', 'wavelet-L_glrlm_GrayLevelNonUniformity', 'wavelet-L_glrlm_GrayLevelNonUniformityNormalized', 'wavelet-L_glrlm_HighGrayLevelRunEmphasis', 'wavelet-L_glrlm_RunLengthNonUniformityNormalized', 'wavelet-L_glrlm_ShortRunEmphasis', 'wavelet-L_glrlm_ShortRunHighGrayLevelEmphasis', 'wavelet-L_glszm_LargeAreaHighGrayLevelEmphasis', 'wavelet-L_glszm_SizeZoneNonUniformityNormalized', 'wavelet-L_glszm_SmallAreaLowGrayLevelEmphasis', 'wavelet-L_glszm_ZoneVariance', 'original_shape2D_Sphericity', 'original_firstorder_TotalEnergy', 'original_glcm_ClusterProminence', 'original_glcm_ClusterTendency', 'original_glcm_JointEntropy', 'original_glcm_MCC', 'original_glcm_MaximumProbability', 'original_glrlm_GrayLevelNonUniformityNormalized', 'original_glszm_LargeAreaLowGrayLevelEmphasis', 'original_glszm_ZoneEntropy', 'wavelet-H_firstorder_Maximum', 'wavelet-H_glcm_Correlation', 'wavelet-H_glcm_DifferenceEntropy', 'wavelet-H_glcm_JointEntropy', 'wavelet-H_glcm_SumEntropy', 'wavelet-H_glrlm_LongRunHighGrayLevelEmphasis', 'wavelet-H_glrlm_RunLengthNonUniformity', 'wavelet-H_glszm_SmallAreaEmphasis', 'wavelet-L_firstorder_10Percentile', 'wavelet-L_firstorder_Energy', 'wavelet-L_firstorder_Entropy', 'wavelet-L_firstorder_InterquartileRange', 'wavelet-L_glcm_Correlation', 'wavelet-L_glcm_InverseVariance', 'wavelet-L_glcm_JointAverage', 'wavelet-L_glrlm_RunLengthNonUniformity', 'original_shape2D_MinorAxisLength', 'original_firstorder_InterquartileRange', 'wavelet-H_glcm_ClusterShade', 'wavelet-H_glrlm_LongRunLowGrayLevelEmphasis', 'wavelet-L_glcm_Imc1', 'wavelet-L_glszm_HighGrayLevelZoneEmphasis', 'wavelet-L_glszm_LowGrayLevelZoneEmphasis', 'wavelet-L_glszm_SmallAreaEmphasis']
)

119

In [3]:
print(consensus_features)

['Unnamed: 0', 'original_shape2D_Elongation_x', 'original_shape2D_MajorAxisLength_x', 'original_shape2D_MaximumDiameter_x', 'original_shape2D_Perimeter_x', 'original_shape2D_PerimeterSurfaceRatio_x', 'original_shape2D_PixelSurface_x', 'original_firstorder_Entropy_x', 'original_firstorder_Range_x', 'original_firstorder_RobustMeanAbsoluteDeviation_x', 'original_glcm_ClusterTendency_x', 'original_glcm_Contrast_x', 'original_glcm_Id_x', 'original_glcm_Idmn_x', 'original_glcm_Idn_x', 'original_glcm_MCC_x', 'original_glcm_MaximumProbability_x', 'original_glcm_SumSquares_x', 'original_glrlm_GrayLevelNonUniformity_x', 'original_glrlm_HighGrayLevelRunEmphasis_x', 'original_glrlm_LongRunLowGrayLevelEmphasis_x', 'original_glrlm_LowGrayLevelRunEmphasis_x', 'original_glrlm_RunEntropy_x', 'original_glrlm_RunLengthNonUniformityNormalized_x', 'original_glrlm_RunPercentage_x', 'original_glrlm_ShortRunHighGrayLevelEmphasis_x', 'original_glszm_GrayLevelNonUniformity_x', 'original_glszm_HighGrayLevelZoneE

In [4]:
len(consensus_features)

243